In [ ]:
pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import string

# Inicializar Faker con localización en Colombia
fake = Faker('es_CO')

# Configurar la semilla aleatoria para reproducibilidad
SEED = 12345
np.random.seed(SEED)
fake.seed_instance(SEED)

# Listas de valores específicos para Colombia
account_types = np.array(['Ahorro', 'Corriente', 'Inversión'])
product_types = np.array(['Préstamo', 'Hipoteca', 'Crédito', 'Tarjeta de Crédito'])
statuses = np.array(['Activo', 'Inactivo', 'Moroso'])
cities = np.array(['Bogotá', 'Medellín', 'Cali', 'Barranquilla', 'Cartagena'])
genders = np.array(['Masculino', 'Femenino'])
identification_types = np.array(['Cédula de ciudadanía', 'Pasaporte', 'Cédula de extranjería'])
payment_types = np.array(['Transferencia', 'Depósito', 'Retiro'])
email_domains = np.array(['gmail.com', 'outlook.com', 'yahoo.com', 'hotmail.com'])

descripciones_personalizadas = np.array([
    "Pago recibido por transferencia interbancaria.",
    "Depósito en cuenta de ahorros.",
    "Retiro de efectivo en cajero Bancolombia.",
    "Pago de factura de servicios públicos.",
    "Transferencia enviada a Nequi.",
    "Pago realizado en supermercado Éxito.",
    "Depósito de nómina en cuenta Bancolombia.",
    "Pago de cuota del crédito hipotecario.",
    "Reembolso por compras en línea.",
    "Pago de matrícula universitaria.",
    "Transferencia entre cuentas de la misma entidad.",
    "Pago de impuestos nacionales DIAN.",
    "Retiro de fondos para inversión en CDT.",
    "Pago recibido por prestación de servicios.",
    "Depósito por devolución de impuestos."
])

# Función para generar identificadores alfanuméricos únicos
def generate_alphanumeric_id(length=10):
    """Genera un identificador alfanumérico único."""
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

# Función para generar números de cuenta numéricos
def generate_numeric_account_number(length=10):
    """Genera un número de cuenta único numérico."""
    return ''.join(random.choices(string.digits, k=length))

# Función para generar teléfonos en formato colombiano (+57 y 10 dígitos)
def generate_phone_number():
    """Genera un número de teléfono colombiano válido con el prefijo +57."""
    prefix = '+57'
    phone_number = np.random.randint(3000000000, 3999999999)  # Números válidos entre 300XXXXXXX y 399XXXXXXX
    return f"{prefix}{phone_number}"

# Función para generar correos electrónicos aleatorios
def generate_email(name):
    """Genera un correo electrónico basado en el nombre y dominios populares."""
    domain = np.random.choice(email_domains)
    email_user = name.lower().replace(" ", ".").replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    return f"{email_user}@{domain}"

# Función para generar clientes
def generate_clients(num):
    cities_selected = np.random.choice(cities, num)  # Seleccionar ciudades
    data = {
        'ClienteID': [generate_alphanumeric_id() for _ in range(num)],
        'Nombre_completo': [fake.name() for _ in range(num)],
        'Ciudad': cities_selected,  # Asignar ciudades a cada cliente
        'Direccion': [f"{fake.street_address()}, {city}" for city in cities_selected],  # Generar dirección con ciudad
        'Telefono': [generate_phone_number() for _ in range(num)],
        'Email': [generate_email(fake.first_name()) for _ in range(num)],
        'Genero': np.random.choice(genders, num),
        'Fecha_nacimiento': [fake.date_of_birth(minimum_age=18, maximum_age=70) for _ in range(num)],
        'Tipo_identificacion': np.random.choice(identification_types, num),
        'Num_identificacion': np.random.randint(1000000000, 1999999999, num),
        'Estado': np.random.choice(statuses, num)
    }
    return pd.DataFrame(data)

# Función para generar cuentas bancarias
def generate_accounts(client_ids, num):
    data = {
        'NumCuenta': [generate_numeric_account_number(12) for _ in range(num)],  # Solo números
        'ClienteID': np.random.choice(client_ids, num),
        'Tipo_cuenta': np.random.choice(account_types, num),
        'Fecha_apertura': [fake.date_this_decade() for _ in range(num)],
        'Saldo': np.round(np.random.uniform(50000, 10000000, num), 2),
        'Limite_credito': np.round(np.random.uniform(100000, 5000000, num), 2)
    }
    return pd.DataFrame(data)

# Función para generar productos financieros
def generate_products(account_ids, num):
    data = {
        'ProductoID': [generate_alphanumeric_id() for _ in range(num)],
        'Tipo_producto': np.random.choice(product_types, num),
        'NumCuenta': np.random.choice(account_ids, num),
        'Monto': np.round(np.random.uniform(1000000, 50000000, num), 2),
        'Tasa_interes': np.round(np.random.uniform(1, 20, num), 2),
        'Fecha_concesion': [fake.date_this_decade() for _ in range(num)],
        'Estado_producto': np.random.choice(statuses, num)
    }
    return pd.DataFrame(data)

# Función para generar transacciones
def generate_transactions(account_ids, num):
    data = {
        'TransaccionID': [generate_alphanumeric_id(14) for _ in range(num)],
        'Fecha_transaccion': [fake.date_this_year() for _ in range(num)],
        'Monto_transaccion': np.round(np.random.uniform(5000, 2000000, num), 2),
        'Estado': np.random.choice(statuses, num),
        'NumCuenta': np.random.choice(account_ids, num),
        'TipoPago': np.random.choice(payment_types, num),
        'Descripcion': np.random.choice(descripciones_personalizadas, num)
    }
    return pd.DataFrame(data)

# Función para generar datos de mora
def generate_mora(accounts_df, num):
    sampled_accounts = accounts_df.sample(num, random_state=SEED)
    data = {
        'MoraID': [generate_alphanumeric_id(10) for _ in range(num)],
        'NumCuenta': sampled_accounts['NumCuenta'].values,
        'ClienteID': sampled_accounts['ClienteID'].values,
        'Monto_pendiente': np.round(np.random.uniform(50000, 500000, num), 2),
        'Dias_mora': np.random.randint(1, 180, num),
        'Fecha_actualizacion': [fake.date_this_month() for _ in range(num)]
    }
    return pd.DataFrame(data)

# Generar los datos
num_clients = 500
clients_df = generate_clients(num_clients)
accounts_df = generate_accounts(clients_df['ClienteID'].tolist(), num_clients * 2)
products_df = generate_products(accounts_df['NumCuenta'].tolist(), num_clients)
transactions_df = generate_transactions(accounts_df['NumCuenta'].tolist(), num_clients * 5)
mora_df = generate_mora(accounts_df, num_clients // 2)
clients_df['Direccion'] = clients_df['Direccion'].replace('\n',' ',regex= True)
clients_df['Direccion'] = clients_df['Direccion'].replace('\t',' ',regex= True)
clients_df['Direccion']= clients_df['Direccion'].replace('"','',regex= True)
clients_df['Direccion'] = clients_df['Direccion'].replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
clients_df['Direccion'] = clients_df['Direccion'].replace("Á", "A").replace("É", "E").replace("Í", "I").replace("Ó", "O").replace("Ú", "U")

# Guardar los datos en archivos CSV
clients_df.to_csv('Clientes_Colombia.csv', index=False,sep= ';')
accounts_df.to_csv('Cuentas_Colombia.csv', index=False)
products_df.to_csv('Productos_Colombia.csv', index=False)
transactions_df.to_csv('Transacciones_Colombia.csv', index=False)
mora_df.to_csv('Mora_Colombia.csv', index=False)


In [ ]:
df = pd.read_csv('/content/Clientes_Colombia.csv',sep= ';')
df

,ClienteID,Nombre_completo,Ciudad,Direccion,Telefono,Email,Genero,Fecha_nacimiento,Tipo_identificacion,Num_identificacion,Estado
0,CD6KSEPTLG,Arturo Fonseca Arenas,Cali,"Transversal 29 Bis A # 7-36, Cali",573305295404,ernesto@gmail.com,Femenino,1967-05-19,Cédula de extranjería,1971913477,Moroso
1,BG2XP9ZJWN,Eduar Fernando Mejía,Medellín,"Avenida Bogotá, D.C. # 66-3, Medellín",573290228084,humberto@hotmail.com,Femenino,1997-04-08,Cédula de extranjería,1598723881,Moroso
2,VPNE1TZTZS,Luis Óscar Londoño,Cartagena,"Calle 67 # 6-1, Cartagena",573703136108,hector@gmail.com,Femenino,2002-06-30,Cédula de extranjería,1513628545,Activo
3,WDDZ01UAY6,Viviana Díaz Daza,Medellín,"Dg. 91B Sur # 30-37 Este, Medellín",573426679317,katherine@outlook.com,Femenino,1982-08-04,Cédula de extranjería,1931521438,Activo
4,BVK6TXD4YI,Mónica Aura Orozco,Cali,"Dg. 98L # 86-99, Cali",573926171764,ana@hotmail.com,Masculino,1974-12-28,Pasaporte,1083463938,Activo
...,...,...,...,...,...,...,...,...,...,...,...
495,EJXKR47WIU,Julio Giraldo,Medellín,"Dg. 15 # 69-7, Medellín",573686287394,jose@yahoo.com,Masculino,1999-04-07,Pasaporte,1058266149,Inactivo
496,E6D55BT554,Ana Lady López,Cali,"Dg. 87 # 57T-3 Sur, Cali",573523187335,ana@yahoo.com,Femenino,1974-09-29,Pasaporte,1173711533,Inactivo
497,G5Y1740OQN,Eliana Caicedo González,Barranquilla,"Diagonal 5P # 6-9 Sur, Barranquilla",573741187057,giovanny@yahoo.com,Masculino,1980-08-11,Cédula de ciudadanía,1670060850,Activo
498,N9PAPUW5A5,Leidy María Suárez,Barranquilla,"Carrera 49 # 3-7 Este, Barranquilla",573058253243,efrain@outlook.com,Femenino,2006-10-08,Cédula de extranjería,1147463274,Moroso


In [ ]:
df = pd.read_csv('/content/Transacciones_Colombia.csv')
df

,TransaccionID,Fecha_transaccion,Monto_transaccion,Estado,NumCuenta,TipoPago,Descripcion
0,K8J07E7YT7X0OY,2024-11-14,1103307.11,Activo,42612431804,Depósito,Reembolso por compras en línea.
1,UBOVRT91N3AXJZ,2024-05-09,1900383.31,Inactivo,257314362749,Retiro,Pago de matrícula universitaria.
2,I357KT7NZBW0OQ,2024-09-02,656869.56,Moroso,703788328485,Depósito,Depósito en cuenta de ahorros.
3,KJA6R4L8TRY09V,2024-06-26,233365.86,Inactivo,64956211270,Depósito,Pago recibido por transferencia interbancaria.
4,94YZNMXTBJWYGE,2024-06-28,1749746.31,Inactivo,734102594765,Depósito,Pago realizado en supermercado Éxito.
...,...,...,...,...,...,...,...
2495,Z05C4658HAM1TG,2024-11-25,1170323.59,Activo,720751652085,Retiro,Pago recibido por transferencia interbancaria.
2496,IY8IQVSK46NQNI,2024-07-07,1475459.97,Inactivo,167152267041,Retiro,Retiro de fondos para inversión en CDT.
2497,EMWUS9EL0ZQH0B,2024-06-07,519818.61,Moroso,15186643255,Depósito,Pago de matrícula universitaria.
2498,145T7UOH5PKPZ9,2024-10-30,1676764.23,Activo,298387889817,Transferencia,Pago de matrícula universitaria.
